# Desafio: Consumo de Dados para Previsão do Tempo das Cidades do Vale do Paraíba.

## Objetivo

Avaliar conhecimentos nas linguagens Python e SQL e na engine de processamento Apache Spark.

## Descrição

Neste desafio, você desenvolverá um notebook que será responsável por extrair dados de previsão do tempo das cidades do Vale do Paraíba, região onde se localiza a Dataside. Para consultar todas as cidades dessa região, utilizaremos a API do IBGE. No caso, basta realizar uma requisição HTTP com o método GET, utilizando a URL abaixo:

```
https://servicodados.ibge.gov.br/api/v1/localidades/mesorregioes/3513/municipios
```

Com esses dados, gerar um data frame e a partir dele uma temp view. Ex: "cities"

Utilizando os nomes das cidades, deverão ser consultados os dados de previsão de tempo para cada cidade. Para realizar essa consulta, poderá ser utilizada qualquer uma das APIs informadas no link abaixo.

[Public APIs - Wather](https://github.com/public-apis/public-apis#weather)

Obs.: Para algumas, pode ser necessário cadastrar-se para acessar sua API Key. Mas nenhuma delas deve precisar cadastrar cartão de crédito ou adicionar qualquer valor monetário para utilizar. Caso alguma solicite, basta optar por outra.

Com os dados consultados, gerar um data frame e partir dele outra temp view. Ex: "forecasts"

Com as temp views geradas, utilizar Spark SQL para criar queries e gerar data frames das seguintes tabelas:

- Tabela 1: dados de previsão do tempo para os próximos cinco dias, para cada data e cidade consultadas. As colunas dessa tabela serão:
    - Cidade
    - CodigoDaCidade
    - Data
    - Regiao
    - Pais
    - Latitude
    - Longigute
    - TemperaturaMaxima
    - TemperaturaMinima
    - TemperaturaMedia
    - VaiChover
    - ChanceDeChuva
    - CondicaoDoTempo
    - NascerDoSol
    - PorDoSol
    - VelocidadeMaximaDoVento
    
    Obs.: Os valores da coluna "VaiChover" deverá ser "Sim" ou "Não". E a coluna "CodigoDaCidade" é o ID retornado junto com os nomes da cidades na API do IBGE.
    Obs.: Dependendo da API utilizada, algumas colunas podem não existir e ficarão em branco. Você deve optar por uma API que traga o maior número de informações possível.

- Tabela 2: quantidade de dias com chuva e sem chuva para os dias consultados, para cada data consultada. Colunas:
    - Cidade
    - QtdDiasVaiChover
    - QtdDiasNaoVaiChover
    - TotalDiasMapeados

Essas tabelas deverão ser exportadas em formado CSV e entregue no final do desafio.

## To Do

[ ] - Consultar municípios do Vale do Paraíba, gerar um data frame e criar uma temp view com esses dados.
[ ] - Consultar dados do tempo para cada município, gerar um data frame e criar uma outra temp view.
[ ] - Utilizar Spark SQL para gerar os data frames das Tabelas 1 e 2.
[ ] - Exportar os data frames para CSV.

## Atenção

- Existe um limite de requisições de 10000 requests por conta cadastrada na m3o.
- Essa API pode retornar cidades de outras regiões que possuem nome semelhante a alguma cidade do Vale do Paraiba. Pode mantê-las ou filtrar para gerar as tabelas apenas com dados de Regiao = Sao Paulo. Fica a seu critério.

## Entregando o desafio

Concluindo todos os passos informados em To Do, basta salvar o arquivo .ipynb do notebook e enviar para a Dataside juntamente com os CSVs das duas tabelas.


In [3]:
import findspark
findspark.init()

import requests
import json
import unidecode
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

spark = SparkSession.builder \
      .master("local[1]") \
      .appName("SparkByExamples.com") \
      .getOrCreate()

In [5]:
# Buscar cidades do Vale do Paraíba
url      = 'https://servicodados.ibge.gov.br/api/v1/localidades/mesorregioes/3513/municipios'
response = requests.get(url)

# Criar data frame com as cidades
df  = spark.createDataFrame(response.json())
df  = df.select('id', 'nome')

# Criar view com as cidades
df.createOrReplaceTempView('cidades')

In [7]:
# df.show(39, truncate=False)

In [17]:
df.show()

+-------+------------------+
|     id|              nome|
+-------+------------------+
|3502507|         Aparecida|
|3503158|            Arapeí|
|3503505|            Areias|
|3504909|           Bananal|
|3508504|          Caçapava|
|3508603|Cachoeira Paulista|
|3509700|  Campos do Jordão|
|3509957|             Canas|
|3510500|     Caraguatatuba|
|3513405|          Cruzeiro|
|3513603|             Cunha|
|3518404|     Guaratinguetá|
|3520202|           Igaratá|
|3520400|          Ilhabela|
|3524402|           Jacareí|
|3524907|          Jambeiro|
|3526308|          Lagoinha|
|3526605|         Lavrinhas|
|3527207|            Lorena|
|3531704|   Monteiro Lobato|
+-------+------------------+
only showing top 20 rows



In [20]:
# Obter a latitude e longitude de todos os endereços
cidades = df.collect()

# Criando as colunas lat e lon
df = df.withColumn('lat', F.lit(None)).withColumn('lon', F.lit(None))

for cidade in cidades:
        nome_cidade = cidade[1]

        url = f"https://nominatim.openstreetmap.org/search?q={nome_cidade}&state=S%C3%A3o+Paulo&country=Brasil&format=json"
        response = requests.get(url)
        info = response.json()
        dic = dict(info[0])

        lat = dic.get('lat')
        lon = dic.get('lon')
        
        # Adicionando as informações no df de cidades
        df = df.withColumn('lat', F.when(F.col('nome') == nome_cidade, lat)
                                   .otherwise(F.col('lat')))
        
        df = df.withColumn('lon', F.when(F.col('nome') == nome_cidade, lon)
                                   .otherwise(F.col('lon')))

In [21]:
df.show()

+-------+------------------+-----------+-----------------+
|     id|              nome|        lat|              lon|
+-------+------------------+-----------+-----------------+
|3502507|         Aparecida| -22.851552|      -45.2340924|
|3503158|            Arapeí|-22.6738889|      -44.4477778|
|3503505|            Areias|-22.5800929|      -44.6975127|
|3504909|           Bananal|-22.6828195|      -44.3221095|
|3508504|          Caçapava| -23.099204|       -45.707645|
|3508603|Cachoeira Paulista| -22.666498|       -45.015384|
|3509700|  Campos do Jordão|-22.7395263|      -45.5912829|
|3509957|             Canas| 43.5515198|        7.0134418|
|3510500|     Caraguatatuba|  -23.62028|        -45.41306|
|3513405|          Cruzeiro|-22.5783685|      -44.9642044|
|3513603|             Cunha|  -23.07753|      -44.9567436|
|3518404|     Guaratinguetá|-22.8057839|      -45.1908926|
|3520202|           Igaratá|-23.2063475|       -46.156934|
|3520400|          Ilhabela| -23.816628|       -45.36868

In [42]:
# Buscar previsão do tempo para as cidades
cidades = df.select('nome', 'lat', 'lon').collect()
df_prev = []

for c in cidades:
    
    nome_cidade = c[0]
    lat         = c[1]
    lon         = c[2]

#     url = f"https://my.meteoblue.com/packages/basic-day_sunmoon?apikey=Fs4qgzAh2CoakwUj&lat={lat}&lon={lon}&format=json&forecast_days=3"
#     response = requests.get(url)
#     info = response.json()

    # Criar data frame com as previsões
    schema = StructType([
        StructField("data",          StringType(), True),
        StructField("cidade",        StringType(), True),
        StructField("estado",        StringType(), True),
        StructField('pais',          StringType(), True),
        StructField('lat',           StringType(), True),
        StructField('lon',           StringType(), True),
        StructField('temp_max',      StringType(), True),
        StructField('temp_min',      StringType(), True),
        StructField('temp_media',    StringType(), True),
        StructField('vai_chover',    StringType(), True),
        StructField("chance_chuva",  StringType(), True),
        StructField("nascer_sol",    StringType(), True),
        StructField("por_sol",       StringType(), True),
        StructField("vel_vento_max", StringType(), True),
    ])

    # Obtendo os valores das variáveis
    data       = info.get('data_day').get('time')
    temp_max   = info.get('data_day').get('temperature_max')
    temp_min   = info.get('data_day').get('temperature_min')
    temp_media = info.get('data_day').get('temperature_mean')
    
    chover = info.get('data_day').get('precipitation_probability')
    chover = [chover[0] > 50, chover[1] > 50, chover[2] > 50]
    
    chance_chuva  = info.get('data_day').get('precipitation_probability')
    nascer_sol    = info.get('data_day').get('sunrise')
    por_sol       = info.get('data_day').get('sunset')
    vel_vento_max = info.get('data_day').get('windspeed_max')
    
    #Criando uma lista que se tornará o dataframe
    for index in range(3):        
        df_prev.append({'data': data[index], 'cidade': nome_cidade, 'estado': 'SP', 'pais': 'Brasil', 
                        'lat': lat, 'lon': lon, 'temp_max': temp_max[index], 'temp_min': temp_min[index], 
                        'temp_media': temp_media[index], 'vai_chover': chover[index], 
                        'chance_chuva': chance_chuva[index], 'nascer_sol': nascer_sol[index], 
                        'por_sol': por_sol[index], 'vel_vento_max': vel_vento_max[index]})

df_previsoes = spark.createDataFrame(df_prev, schema)

# Criar view com as previsões
df_previsoes.createOrReplaceTempView('previsoes')

In [52]:
# Criar DF da Tabela 1
tabela1 = df_previsoes

In [78]:
df_previsoes.toPandas()

,data,cidade,estado,pais,lat,lon,temp_max,temp_min,temp_media,vai_chover,chance_chuva,nascer_sol,por_sol,vel_vento_max
0,2022-09-19,Aparecida,SP,Brasil,-22.851552,-45.2340924,21.37,16.42,19.11,false,0,05:52,17:56,2.71
1,2022-09-20,Aparecida,SP,Brasil,-22.851552,-45.2340924,23.34,18.59,21.07,true,72,05:51,17:56,1.65
2,2022-09-21,Aparecida,SP,Brasil,-22.851552,-45.2340924,22.16,19.88,20.79,true,71,05:50,17:57,1.77
3,2022-09-19,Arapeí,SP,Brasil,-22.6738889,-44.4477778,21.37,16.42,19.11,false,0,05:52,17:56,2.71
4,2022-09-20,Arapeí,SP,Brasil,-22.6738889,-44.4477778,23.34,18.59,21.07,true,72,05:51,17:56,1.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,2022-09-20,Tremembé,SP,Brasil,-22.95714,-45.547526,23.34,18.59,21.07,true,72,05:51,17:56,1.65
113,2022-09-21,Tremembé,SP,Brasil,-22.95714,-45.547526,22.16,19.88,20.79,true,71,05:50,17:57,1.77
114,2022-09-19,Ubatuba,SP,Brasil,-23.433162,-45.083415,21.37,16.42,19.11,false,0,05:52,17:56,2.71
115,2022-09-20,Ubatuba,SP,Brasil,-23.433162,-45.083415,23.34,18.59,21.07,true,72,05:51,17:56,1.65


In [86]:
# Criar DF da Tabela 2
# tabela2 = df_previsoes.
# df_previsoes.groupBy('cidade', 'vai_chuver').count().orderBy('cidade').show()

# df_vc -> dataframe com a quantidade de vezes que vai chover por cidade
df_vc = df_previsoes.where(F.col('vai_chover') == True) \
                    .groupBy('cidade', 'vai_chover') \
                    .agg(F.count('vai_chover').alias('q_vai_chover')) \
                    .orderBy('cidade') \
                    .select('cidade', 'q_vai_chover')\

# df_nvc -> dataframe com a quantidade de vezes que não vai chover por cidade
df_nvc = df_previsoes.where(F.col('vai_chover') == False) \
                     .groupBy('cidade', 'vai_chover') \
                     .agg(F.count('vai_chover').alias('q_nao_vai_chover')) \
                     .orderBy('cidade') \
                     .select(F.col('cidade').alias('cidade2'), 'q_nao_vai_chover')\

df_tab2 = df_vc.join(df_nvc,
                     df_vc['cidade'] == df_nvc['cidade2'],
                     'left')

df_tab2 = df_tab2.drop('cidade2')

df_tab2.show()

# tabela2 = df_previsoes.select(F.count(F.col('vai_chuver') == 'true').alias('vai_chover'))
# tabela2.show()

+------------------+------------+----------------+
|            cidade|q_vai_chover|q_nao_vai_chover|
+------------------+------------+----------------+
|         Aparecida|           2|               1|
|            Arapeí|           2|               1|
|            Areias|           2|               1|
|           Bananal|           2|               1|
|Cachoeira Paulista|           2|               1|
|  Campos do Jordão|           2|               1|
|             Canas|           2|               1|
|     Caraguatatuba|           2|               1|
|          Caçapava|           2|               1|
|          Cruzeiro|           2|               1|
|             Cunha|           2|               1|
|     Guaratinguetá|           2|               1|
|           Igaratá|           2|               1|
|          Ilhabela|           2|               1|
|           Jacareí|           2|               1|
|          Jambeiro|           2|               1|
|          Lagoinha|           

In [ ]:
# Exportar CSVs
# TODO